### Univariate Time Series Forecasting

In [1]:
import pandas as pd
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import *
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from numpy import array

df_carsale = pd.read_csv("monthly-car-sales.csv")

Using TensorFlow backend.


In [2]:
df_carsale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 2 columns):
Month    108 non-null object
Sales    108 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


In [3]:
df_carsale.head(n=10)

,Month,Sales
0,1960-01,6550
1,1960-02,8728
2,1960-03,12026
3,1960-04,14395
4,1960-05,14587
5,1960-06,13791
6,1960-07,9498
7,1960-08,8251
8,1960-09,7049
9,1960-10,9545


In [5]:
train = df_carsale.Sales.values
train = [train[i] - train[i - 12] for i in range(12, len(train))]
df_train=array(train)

In [10]:
df_train[:5]

array([ 687,  646, -189, -611, 1339], dtype=int64)

In [ ]:
# Timeseries plot
sns.lineplot(x="Month", y="Sales", data=df_carsale)

In [ ]:
def supervised_dataframe(df_tmp, lag):
    for i in range(lag):
        df_tmp["Sales_%s"%('%02d' %i)] = df_tmp.Sales.shift(i+1)
    df_tmp.dropna(inplace=True)
    return df_tmp

In [ ]:
lags = 36
df_carsale = df_carsale.drop(['Month'], axis=1)
df_carsale_lagged = supervised_dataframe(df_carsale, lags)
df_carsale_lagged = df_carsale_lagged.reindex_axis(sorted(df_carsale_lagged.columns, reverse=True), axis=1)
df_carsale_lagged.head() 

In [ ]:
x_train = df_carsale_lagged.drop(['Sales'], axis=1).values
y_train = df_carsale_lagged.Sales.values
# Partitioning the training data
x_trn, x_test, y_trn, y_test = train_test_split(x_train, y_train, test_size=0.11, random_state=1, shuffle=False)

In [ ]:
# define model
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=36))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1))
model.summary()

model.compile(loss='mse', optimizer='adam')

In [ ]:
history = model.fit(x_trn, y_trn, epochs=100, verbose=1)

In [ ]:
pyplot.plot(history.history['loss'])
#pyplot.plot(history.history[ val_loss ])
#pyplot.title( model train vs validation loss )
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
#pyplot.legend([ train , validation ], loc= upper right )
pyplot.show()

In [ ]:
yhat = model.predict(x_test, verbose=0)

In [ ]:
yhat

In [ ]:
test_error_rate = model.evaluate(x_test, y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# root mean squared error or rmse
print(sqrt(mean_squared_error(y_test, yhat)))

In [ ]:
# define model
model_ls = Sequential()
model_ls.add(LSTM(50, activation='relu', input_shape=(36, 1)))
model_ls.add(Dense(50, activation='relu'))
model_ls.add(Dense(1))

model_ls.summary()

model_ls.compile(loss='mse', optimizer='adam')

In [ ]:
train_x = array(x_trn)
train_x = x_trn.reshape((x_trn.shape[0], x_trn.shape[1], 1))
history_ls = model_ls.fit(train_x, y_trn, epochs=200, verbose=1)

In [ ]:
pyplot.plot(history_ls.history['loss'])
#pyplot.plot(history.history[ val_loss ])
#pyplot.title( model train vs validation loss )
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
#pyplot.legend([ train , validation ], loc= upper right )
pyplot.show()

In [ ]:
test_x = array(x_test)
test_x = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
test_error_rate_ls = model_ls.evaluate(test_x, y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(sqrt(test_error_rate_ls)))